In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [21]:
# loading the dataset"
train = pd.read_csv("Doceree-HCP_Train.csv", encoding='unicode_escape', on_bad_lines='skip')


In [82]:
test = pd.read_csv("Doceree-HCP_Test.csv", encoding='unicode_escape', on_bad_lines='skip')

# DATA CLEANING

## Applying on the Train Set

In [22]:
train.shape

(113937, 14)

In [23]:
train.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81624
IS_HCP                 1
dtype: int64

In [24]:
# Extracting rows where IS_HCP is Null
train.loc[train["IS_HCP"].isnull()]

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
73790,74791,Mobile,9,47.147.27.131,e180c16e-a393-48aa-ac14-7353faad95ee,Hacienda Heights,91745.0,Mozilla/5.0 (iPhone; CPU iPhone OS 15_6_1 like...,Online Medical Journal,Website,https://www.2minutemedicine.com/the-accord-tri...,Anemia|Delivery|Oncology|Medicine|Diabetes Mel...,NaN,NaN


In [27]:
# Now as we know our target columns are IS_NULL and TAXOMONY we try to remove the outliers
# Calculating the outliers
Q1 = train['IS_HCP'].quantile(0.25)
Q3 = train['IS_HCP'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR

# Removing the outliers
train = train[(train['IS_HCP'] > lower_limit) & (train['IS_HCP'] < upper_limit)]

In [28]:
#now again checking null values after outliers
train.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            6359
USERZIPCODE         4592
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81623
IS_HCP                 0
dtype: int64

In [45]:
# Now we will work on the missing values of USER CITY adn USERZIP Columns


# Uupdating the values for USERZIPCODE (where both USERZIPCODE AND USERCITY have null values)
condition1= (train["USERCITY"].isnull())
condition2= (train["USERZIPCODE"].isnull())
train.loc[condition1 & condition2, "USERZIPCODE"] = 0
train.loc[condition1 & condition2, "USERCITY"] = 'Not_Specified'

# Now updating the zip code of the 2 remaining missing userzip rows
index = train[train["USERCITY"]=="Bynumville"].index
train.loc[index, "USERZIPCODE"]='65281'

In [47]:
train.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            1769
USERZIPCODE            0
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81623
IS_HCP                 0
dtype: int64

In [55]:
# Creating a copy of train dataframe   
temp = train.copy()

In [56]:
temp.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY            1769
USERZIPCODE            0
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81623
IS_HCP                 0
dtype: int64

In [60]:
# Now we will be handling the USERCITY we will use the Geopy library to extract the city names with the help of zip code 
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geocoder")

for index, row in temp.iterrows():
    if pd.isnull(row["USERCITY"]):
        zipcode = row["USERZIPCODE"]
        location = geolocator.geocode(zipcode)
        if location is not None:
            address = location.address
            address_parts = address.split(",")
            if len(address_parts) >= 3:
                city = address_parts[-3].strip()
                temp.at[index, "USERCITY"] = city


In [61]:
temp.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY               0
USERZIPCODE            0
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81623
IS_HCP                 0
dtype: int64

In [59]:
#converting userzipcode into int type in temp data fram  
temp["USERZIPCODE"]= temp["USERZIPCODE"].astype(int)

In [62]:
# Updating our training dataframe after changes in temp dataframe
train = temp.copy()

train.isnull().sum()

ID                     0
DEVICETYPE             0
PLATFORM_ID            0
BIDREQUESTIP           0
USERPLATFORMUID        4
USERCITY               0
USERZIPCODE            0
USERAGENT              2
PLATFORMTYPE           0
CHANNELTYPE            0
URL                    0
KEYWORDS               0
TAXONOMY           81623
IS_HCP                 0
dtype: int64

In [63]:
# Now updating the remaining NULL ROWS by providing empyt string

train.loc[train["USERPLATFORMUID"].isnull()]= ' '
train.loc[train["USERAGENT"].isnull()]= ' '
train.loc[train["TAXONOMY"].isnull()]= ' '


In [64]:
# removing extra spaces 
train.replace(' ','_', regex=True, inplace=True  )

In [65]:
train.isnull().sum()

ID                 0
DEVICETYPE         0
PLATFORM_ID        0
BIDREQUESTIP       0
USERPLATFORMUID    0
USERCITY           0
USERZIPCODE        0
USERAGENT          0
PLATFORMTYPE       0
CHANNELTYPE        0
URL                0
KEYWORDS           0
TAXONOMY           0
IS_HCP             0
dtype: int64

In [66]:
train.to_csv("cleantrain.csv", index=False)

## Applying on TEST SET

In [83]:
test.isnull().sum()

ID                    0
DEVICETYPE            0
PLATFORM_ID           0
BIDREQUESTIP          0
USERPLATFORMUID       0
USERCITY           1559
USERZIPCODE        1115
USERAGENT             1
PLATFORMTYPE          0
CHANNELTYPE           0
URL                   0
KEYWORDS              0
dtype: int64

In [84]:
# We will handle the USERCITY AND USERZIP CODE 

# First assigning values where both values are NUll
test.loc[test['USERCITY'].isnull() & test['USERZIPCODE'].isnull(), 'USERZIPCODE'] = 0
test.loc[test['USERCITY'].isnull() & test['USERZIPCODE'].isnull(), 'USERCITY'] = 'Not_Specified'


In [85]:
test.isnull().sum()

ID                    0
DEVICETYPE            0
PLATFORM_ID           0
BIDREQUESTIP          0
USERPLATFORMUID       0
USERCITY           1559
USERZIPCODE           0
USERAGENT             1
PLATFORMTYPE          0
CHANNELTYPE           0
URL                   0
KEYWORDS              0
dtype: int64

In [91]:
# Now we will get the USERCITY which are empty fromm the USERZIP CODES, similarly we create a temporary data frame to updata it 
temp = test.copy()

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geocoder")

for index, row in temp.iterrows():  
    if pd.isnull(row["USERCITY"]):
        zipcode = row["USERZIPCODE"]
        location = geolocator.geocode(zipcode)
        if location is not None:
            address = location.address
            address_parts = address.split(",")
            if len(address_parts) >= 3:
                city = address_parts[-3].strip()
                temp.at[index, "USERCITY"] = city


temp.isnull().sum()


ID                 0
DEVICETYPE         0
PLATFORM_ID        0
BIDREQUESTIP       0
USERPLATFORMUID    0
USERCITY           3
USERZIPCODE        0
USERAGENT          1
PLATFORMTYPE       0
CHANNELTYPE        0
URL                0
KEYWORDS           0
dtype: int64

In [92]:
temp.isnull().sum()


ID                 0
DEVICETYPE         0
PLATFORM_ID        0
BIDREQUESTIP       0
USERPLATFORMUID    0
USERCITY           3
USERZIPCODE        0
USERAGENT          1
PLATFORMTYPE       0
CHANNELTYPE        0
URL                0
KEYWORDS           0
dtype: int64

In [87]:
temp["USERZIPCODE"]= temp["USERZIPCODE"].astype(int)

In [94]:
temp.loc[temp['USERCITY'].isnull()]

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS
5122,120623,Desktop,3,12.111.139.130,ddda887b-878b-43d3-b665-ae2bcb674478,NaN,97227.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/drugs/anti-depressants/,Pharmacology|Liver Dysfunction|Elderly|Liver|C...
6589,122090,Desktop,3,12.111.139.130,ddda887b-878b-43d3-b665-ae2bcb674478,NaN,97227.0,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://globalrph.com/drugs/anti-depressants/,Pharmacology|Liver Dysfunction|Elderly|Liver|C...
15394,130895,Desktop,3,172.5.156.181,34182a0b-413e-4da7-b760-41dcc3cee1a7,NaN,95403.0,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,Online Medical Journal,Website,https://globalrph.com/medcalcs/protein-require...,Gastrointestinal Tract|Female|Oncology|Medicin...


In [80]:
test = temp.copy()

In [71]:
# Now updating the remaining NULL ROWS by providing empyt string

test.loc[test["USERAGENT"].isnull()]= ' '
test.loc[test["KEYWORDS"].isnull()]= ' '


In [81]:
test.isnull().sum()

ID                    0
DEVICETYPE            0
PLATFORM_ID           0
BIDREQUESTIP          0
USERPLATFORMUID       0
USERCITY           1515
USERZIPCODE           0
USERAGENT             1
PLATFORMTYPE          0
CHANNELTYPE           0
URL                   0
KEYWORDS              0
dtype: int64

In [111]:
test.to_csv('cleantest.csv', index=False)

In [113]:
train.columns

Index(['ID', 'DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERPLATFORMUID',
       'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE',
       'URL', 'KEYWORDS', 'TAXONOMY', 'IS_HCP'],
      dtype='object')

In [118]:
train.sample()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS,TAXONOMY,IS_HCP
103516,104230,Desktop,2.0,68.129.74.33,46aee387-36ee-4f61-a458-dddba65e7a79,Lynbrook,11563,Mozilla/5.0_(Windows_NT_10.0;_Win64;_x64)_Appl...,Online_Medical_Journal,Website,https://www.medicalbag.com/,Genetic|Large|Autoimmune|Health|False|Abortion...,207R00000X,1.0


In [119]:
test.columns        

Index(['ID', 'DEVICETYPE', 'PLATFORM_ID', 'BIDREQUESTIP', 'USERPLATFORMUID',
       'USERCITY', 'USERZIPCODE', 'USERAGENT', 'PLATFORMTYPE', 'CHANNELTYPE',
       'URL', 'KEYWORDS'],
      dtype='object')

In [120]:
test.sample()

,ID,DEVICETYPE,PLATFORM_ID,BIDREQUESTIP,USERPLATFORMUID,USERCITY,USERZIPCODE,USERAGENT,PLATFORMTYPE,CHANNELTYPE,URL,KEYWORDS
1779,117280,Desktop,2.0,68.132.98.211,d76f7c0b-1a64-4d4b-b36d-86c832be8837,Deer Park,11729,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,Online Medical Journal,Website,https://www.gastroenterologyadvisor.com/,Transplantation|Technology|Chronic|Neoplasms|U...
